# Creating a flowmap with image reference detretching

In [6]:
#import dependencies

import sys
import os, os.path
sys.path.append(os.path.abspath(".."))  # Adjust path to package root

from abstraction import calc_offset_vectors, destretch_files
from utility import get_fits_paths, IndexSchema
from reference_method import RollingWindow

# third party
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

# internal
import abstraction
import algorithm
from astropy.io import fits
from utility import IndexSchema, get_fits_paths, load_image_data
from examples.fits_to_mp4 import fits_to_mp4, data_to_mp4
from reference_method import ExternalRefs, RollingWindow, WindowEdgeBehavior

# get the filepaths of the timeseries we want to destretch
files_dir = os.path.abspath(os.path.join(".", "media", "large"))
files = get_fits_paths(files_dir)
files = files[610:722]

ref_means_dir = os.path.abspath(os.path.join(files_dir, "f_ref_means"))
offs_dir = os.path.abspath(os.path.join(files_dir, "f_offs"))
flow_offs_dir = os.path.abspath(os.path.join(files_dir, "f_flow_offs"))
destr_files_dir = os.path.abspath(os.path.join(files_dir, "f_destretched"))

window_size: int = 25
kernel_sizes: list[int] = [128, 64]

In [ ]:


# ref_means = Rolling mean of image data
ref_means = abstraction.calc_rolling_mean(
    files, 
    ref_means_dir, 
    "ref_mean",
    window_left=window_size,
    window_right=window_size
)

# offs = offsets from files vs ref_means
offs = abstraction.calc_offset_vectors(
    files, 
    offs_dir,
    "off",
    ref_method=ExternalRefs(files, ref_means),
    kernel_sizes=kernel_sizes
)

# flow_offs = Rolling mean of offs
flow_offs = abstraction.calc_rolling_mean(
    offs, 
    flow_offs_dir,
    "flow_off",
    window_left=window_size,
    window_right=window_size,
    end_behavior=WindowEdgeBehavior.KEEP_RANGE
)

# destretch with the given
destr_files = abstraction.destretch_files(
    files, 
    offs,
    destr_files_dir,
    "destretched"
)

writing /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_ref_means/ref_mean000.avg.fits..
writing /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_ref_means/ref_mean001.avg.fits..
writing /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_ref_means/ref_mean002.avg.fits..
writing /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_ref_means/ref_mean003.avg.fits..
writing /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_ref_means/ref_mean004.avg.fits..
writing /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_ref_means/ref_mean005.avg.fits..
writing /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_ref_means/ref_mean006.avg.fits..
writing /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_ref_means/ref_mea

In [10]:
# the flow map can be visualized by showing flow offs in sequence

vid_flow_file = os.path.abspath(os.path.join(files_dir, "f_flowmap.mp4"))
vid_off_file = os.path.abspath(os.path.join(files_dir, "f_offmap.mp4"))
vid_image_file = os.path.abspath(os.path.join(files_dir, "f_original.mp4"))
vid_destretched_file = os.path.abspath(os.path.join(files_dir, "f_destretched.mp4"))

fits_to_mp4(
    files, 
    vid_image_file, 
    30, "copper",
    IndexSchema.XY,
    relative_max=1.25
)

fits_to_mp4(
    get_fits_paths(destr_files_dir), 
    vid_destretched_file, 
    30, "copper",
    IndexSchema.XY,
    relative_max=1.25
)

fits_to_mp4(
    get_fits_paths(offs_dir), 
    vid_off_file, 
    30, None, 
    normal_mode=True
)

fits_to_mp4(
    get_fits_paths(flow_offs_dir), 
    vid_flow_file, 
    30, None, 
    normal_mode=True
)

Video saved to /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_original.mp4, with 112 frames
Video saved to /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_destretched.mp4, with 112 frames
Video saved to /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_offmap.mp4, with 112 frames
Video saved to /home/tteck/Documents/development/flct_destretch/flctdestretch/examples/media/large/f_flowmap.mp4, with 112 frames
